In [13]:
import tkinter as tk
from tkinter import ttk, messagebox
import psutil

# Function to get process list with parent-child relationship
def get_process_list():
    processes = []
    for proc in psutil.process_iter(['pid', 'name', 'memory_percent', 'cpu_percent', 'ppid']):
        try:
            # Get process info
            pinfo = [
                proc.info['pid'],
                proc.info['ppid'],
                proc.info['name'],
                proc.info['cpu_percent'],
                proc.info['memory_percent']
            ]
            processes.append(pinfo)
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return processes

# Function to kill a process
def kill_process():
    selection = tree.focus()
    if selection:
        process = tree.item(selection)['values']
        pid = int(process[0])  # Extract PID, which should now correctly be the first item
        try:
            p = psutil.Process(pid)
            
            # Preview the impact
            cpu_usage = p.cpu_percent(interval=0.1)
            memory_usage = p.memory_percent()
            message = (
                f"Terminating Process: {p.name()} (PID: {pid})\n"
                f"Current CPU Usage: {cpu_usage}%\n"
                f"Current Memory Usage: {memory_usage}%\n\n"
                "Are you sure you want to terminate this process?"
            )
            
            if messagebox.askyesno("Confirm Termination", message):
                p.terminate()
                print(f"Process with PID {pid} terminated successfully.")
                populate_tree()  # Refresh the tree after termination
                
        except psutil.NoSuchProcess:
            print(f"Error: Process with PID {pid} does not exist.")
        except psutil.AccessDenied:
            print(f"Error: Access denied. Unable to terminate process with PID {pid}.")

# Function to populate the treeview with processes
def populate_tree():
    for i in tree.get_children():
        tree.delete(i)
    processes = get_process_list()
    process_dict = {proc[0]: proc for proc in processes}
    inserted = set()
    
    def insert_process(proc):
        pid = proc[0]
        ppid = proc[1]
        if pid in inserted:
            return
        if ppid == 0 or ppid not in process_dict:
            tree.insert('', 'end', iid=pid, values=proc)
        else:
            parent_iid = str(ppid)
            if parent_iid not in inserted:
                insert_process(process_dict[ppid])
            tree.insert(parent_iid, 'end', iid=pid, values=proc)
        inserted.add(pid)
    
    for proc in processes:
        insert_process(proc)

# Create main window
root = tk.Tk()
root.title('Process Manager')

# Create Treeview to show processes
columns = ('PID', 'PPID', 'Name', 'CPU Usage (%)', 'Memory Usage (%)')
tree = ttk.Treeview(root, columns=columns, show='headings')
for col in columns:
    tree.heading(col, text=col)
tree.column('PID', width=100)
tree.column('PPID', width=100)
tree.column('Name', width=200)
tree.column('CPU Usage (%)', width=150)
tree.column('Memory Usage (%)', width=150)
tree.pack(expand=True, fill=tk.BOTH)

# Button to kill process
kill_button = ttk.Button(root, text="Kill Process", command=kill_process)
kill_button.pack(fill=tk.X)

# Button to refresh process list
refresh_button = ttk.Button(root, text="Refresh", command=populate_tree)
refresh_button.pack(fill=tk.X)

# Initially populate the tree
populate_tree()

# Start the GUI
root.mainloop()
